# FY 2018 Endangered Species Expenditures

### Data Source:

2018 Endangered Species Expenditures Data was collected from the following public report on pages 8-104:
https://www.fws.gov/sites/default/files/documents/endangered-and-threatened-species-expenditures-fiscal-year-2018.pdf

Note: This does not include expenditures for land acquistion

### Module Imports:

In [1]:
import pandas as pd
import numpy as np
import tabula as tb
import re
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
sns.set(rc = {'figure.figsize':(15,8)}, color_codes=True)

### Data Cleaning:

First I converted table 1 in the pdf to a csv file:

In [2]:
page_list = list(range(8, 105))
file = 'endangered-and-threatened-species-expenditures-fiscal-year-2018.pdf'
tb.convert_into(file, "expenditures2018.csv", pages = page_list, output_format ="csv", stream = True)
df = pd.read_csv('expenditures2018.csv')
df.head()

ParserError: Error tokenizing data. C error: Expected 9 fields in line 344, saw 10


In [ ]:
df.tail()

After viewing a snapshot of the data, I realized that there were inconsistencies columns and rows needed to be manually cleaned.

In [ ]:
# after cleaning renamed data to 'cleaned_expenditures2018.csv'
df2018 = pd.read_csv('cleaned_expenditures2018.csv')
print(df2018.columns)
df2018.head()

Renaming columns for consistency across dataframes & splitting combined columns:

In [ ]:
df2018 = df2018.rename(columns={'Species ':'Species',
                                'Group Name':'Group',
                                'FWS Total':'FWS 2018',
                                'Other Fed':'Other Fed 2018',
                                'States Total':'States 2018', 
                                'Species Total':'Total 2018'})
# split column and add new columns to df
df2018[['Inverted Common Name','Scientific Name',
        'Noname1', 'Noname2', 'Noname3', 'Noname4' ]] = df2018['Species'].str.split('(', expand=True)
df2018[['Scientific Name','Area', 'Noname5', 'Noname6', 'Noname7']] = df2018['Scientific Name'].str.split('-', expand=True)

#drop extra columns
df2018 = df2018.drop(['Rank','Federal Total','Species','Noname1',
                      'Noname2','Noname3','Noname4','Noname5',
                      'Noname6','Noname7'], axis = 1)
df2018.head()

In order to perform EDA, needed to remove symbols and change data types:

In [ ]:
#remove unnecessary symbols
df2018['Scientific Name'] = df2018['Scientific Name'].str.replace('[()=]', '', regex=True)

#changed data type to integer for analysis
df2018['Total 2018'] = df2018['Total 2018'].str.replace('[/$,]', '', regex=True).astype(int)
df2018['States 2018'] = df2018['States 2018'].str.replace('[/$,]', '', regex=True).astype(int)
df2018['FWS 2018'] = df2018['FWS 2018'].str.replace('[/$,]', '', regex=True).astype(int)
df2018['Other Fed 2018'] = df2018['Other Fed 2018'].str.replace('[/$,]', '', regex=True).astype(int)

# display the dataframe
df2018.head()

In [ ]:
print(df2018.shape)
# checking the stats for the expenditures
df2018.describe()

In [ ]:
df2018.groupby("Group").size()

In [ ]:
#Changing the order of the columns displayed
df2018 = df2018[['Group','Status','Scientific Name','Inverted Common Name',
        'FWS 2018','Other Fed 2018','States 2018','Total 2018','Area']]

#sorting the values by 'Group' and resetting the index
df2018.sort_values(by=['Group'], inplace=True)
df2018 = df2018.reset_index()
df2018 = df2018.drop(['index'], axis = 1)

df2018.head()

Now that the index is reset, I can use the group count totals from groupby("Group").size() to drop the group subtotal rows that could skew my analysis.

In [ ]:
# to preserve df2017, I made a copy
df2018_copy = df2018

# performing the drop on the copy only
df2018_copy = df2018_copy.drop(index=[
    38,50,161,274,279,296,323,362,543,
    1433,1522,1525,1647,1648,1709,1760,1761]) 

# checking to make sure that only the subtotals were dropped
df2018_copy['Group'].value_counts()

Now that the subtotal rows are dropped:

In [ ]:
#Checking the null values.
print(df2018_copy.isnull().sum())


In [ ]:
#checking display before exporting
df2018_copy.head()

Now the dataframe is ready for analysis. I convert the copy (without subtotals) to a new csv

In [ ]:
df2018_copy.to_csv('esa_expenditures2018.csv', index=False)